In [1]:
from datetime import datetime
from os.path import join
from os import listdir
import re #for camel case conversion
import json
import ast

import numpy as np
import pandas as pd

import torch
from sklearn.preprocessing import LabelEncoder
import os
from os.path import join
from datetime import datetime
from os import listdir
import re #for camel case conversion
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

os.environ['LDA_name'] = 'num-directstr_thr-0_tn-400'

from extract.feature_extraction.topic_features_LDA import extract_topic_features
from extract.feature_extraction.sherlock_features import extract_sherlock_features
from utils import get_valid_types
from model import models_sherlock
from model.torchcrf import CRF

import altair as alt
alt.renderers.enable('default')

/home/senn/virtualenvs/col2type/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/home/senn/virtualenvs/col2type/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more det

RendererRegistry.enable('default')

In [2]:
def camel_case(s):
  s = re.sub(r"(_|-)+", " ", s).title().replace(" ", "")
  return ''.join([s[0].lower(), s[1:]])

def sherlock_case(s):
    s = re.sub(r"(_|-)+", " ", s).title().replace(" ", "")
    s = ''.join([s[0].lower(), s[1:]])
    s = ''.join(map(lambda x: x if x.islower() else " "+x, s))
    return s

In [3]:
path_raw = '../../sherlock/data/data/raw'
path_processed = '../../sherlock/data/data/processed'
path_out_true_types = '../../combined/results/true_types'
path_out_predictions = '../../combined/results/predictions'
path_out_temp = '../results/sato_sherlock'

In [4]:
# test_values = pd.read_parquet(join(path_raw, "test_values.parquet"))
# train_values = pd.read_parquet(join(path_raw, "train_values.parquet"))
validation_values = pd.read_parquet(join(path_raw, "val_values.parquet"))

# print(len(test_values))
# print(len(train_values))
print(len(validation_values['values']))
# test_labels = pd.read_parquet(join(path_raw, "test_labels.parquet"))
# train_labels = pd.read_parquet(join(path_raw, "train_labels.parquet"))
# validation_labels = pd.read_parquet(join(path_raw, "val_labels.parquet"))

# test_processed = pd.read_parquet(join(path_processed, "test.parquet"))
# train_processed = pd.read_parquet(join(path_processed, "train.parquet"))
# validation_processed = pd.read_parquet(join(path_processed, "validation.parquet"))

137353


In [5]:
# values = pd.concat([test_values, train_values, validation_values], ignore_index = True, axis = 0)
# labels = pd.concat([test_labels, train_labels, validation_labels], ignore_index = True, axis = 0)
# processed = pd.concat([test_processed, train_processed, validation_processed], ignore_index = True, axis = 0)

In [6]:
# values
# print(type(values)) 
# print(len(values))
# print(len(processed))
# labels.head(10)
# print(len(labels))
# labels.to_parquet(join(path_out_true_types, "sherlock.parquet"))

### Sherlock

In [7]:
# model = SherlockModel();
# model.model_files_directory = '../../../sherlock/model_files'
# model.initialize_model_from_json(with_weights=True, model_id="sherlock");

In [8]:
# predicted_labels = model.predict(processed, "sherlock")
# predicted_labels = pd.DataFrame(predicted_labels)

In [9]:
# predicted_labels.columns = ['type']
# predicted_labels.to_parquet(join(path_out_predictions, "sherlock_sherlock.parquet"))

### Sato

In [10]:
TYPENAME = os.environ['TYPENAME']
valid_types = get_valid_types(TYPENAME)
print(valid_types)
label_enc = LabelEncoder()
label_enc.fit(valid_types)

MAX_COL_COUNT = 10
topic_dim = 400
pre_trained_loc = './pretrained_sato'
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print("PyTorch device={}".format(device))
device = torch.device('cpu')

['address', 'affiliate', 'affiliation', 'age', 'album', 'area', 'artist', 'birthDate', 'birthPlace', 'brand', 'capacity', 'category', 'city', 'class', 'classification', 'club', 'code', 'collection', 'command', 'company', 'component', 'continent', 'country', 'county', 'creator', 'credit', 'currency', 'day', 'depth', 'description', 'director', 'duration', 'education', 'elevation', 'family', 'fileSize', 'format', 'gender', 'genre', 'grades', 'isbn', 'industry', 'jockey', 'language', 'location', 'manufacturer', 'name', 'nationality', 'notes', 'operator', 'order', 'organisation', 'origin', 'owner', 'person', 'plays', 'position', 'product', 'publisher', 'range', 'rank', 'ranking', 'region', 'religion', 'requirement', 'result', 'sales', 'service', 'sex', 'species', 'state', 'status', 'symbol', 'team', 'teamName', 'type', 'weight', 'year']


In [11]:
feature_group_cols = {}
sherlock_feature_groups = ['char', 'word', 'par', 'rest']
for f_g in sherlock_feature_groups:
    feature_group_cols[f_g] = list(pd.read_csv(join(os.environ['BASEPATH'],
                                          'configs', 'feature_groups', 
                                          "{}_col.tsv".format(f_g)),
                                           sep='\t', header=None, 
                                           index_col=0)[1])

In [12]:
pad_vec = lambda x: np.pad(x, (0, topic_dim - len(x)),
                    'constant',
                    constant_values=(0.0, 1/topic_dim))

In [13]:
classifier = models_sherlock.build_sherlock(sherlock_feature_groups, num_classes=len(valid_types), topic_dim=topic_dim, dropout_ratio=0.35)
#classifier.load_state_dict(torch.load(join(pre_trained_loc, 'sherlock_None.pt'), map_location=device))
model = CRF(len(valid_types) , batch_first=True).to(device)
#model.load_state_dict(torch.load(join(pre_trained_loc, 'model.pt'), map_location=device))

loaded_params = torch.load(join(pre_trained_loc, 'model.pt'), map_location=device)
classifier.load_state_dict(loaded_params['col_classifier'])
model.load_state_dict(loaded_params['CRF_model'])

classifier.eval()
model.eval()

CRF(num_tags=78)

In [14]:
def extract(df):

    df_dic = {'df':df, 'locator':'None', 'dataset_id':'None'}
    feature_dic = {}
    n = df.shape[1]

    # topic vectors
    topic_features = extract_topic_features(df_dic)
    topic_vec = pad_vec(topic_features.loc[0,'table_topic'])
    feature_dic['topic'] = torch.FloatTensor(np.vstack((np.tile(topic_vec,(n,1)), np.zeros((MAX_COL_COUNT - n, topic_dim)))))


    # sherlock vectors
    sherlock_features = extract_sherlock_features(df_dic)
    for f_g in feature_group_cols:
        temp = sherlock_features[feature_group_cols[f_g]].to_numpy()
        temp = np.vstack((temp, np.zeros((MAX_COL_COUNT - n, temp.shape[1])))).astype('float')
        temp = np.nan_to_num(temp)
        feature_dic[f_g] = torch.FloatTensor(temp)

    # dictionary of features, labels, masks
    return feature_dic, np.zeros(MAX_COL_COUNT), torch.tensor([1]*n + [0]*(MAX_COL_COUNT-n), dtype=torch.uint8)

In [15]:
def evaluate(df):

    feature_dic, labels, mask = extract(df)

    emissions = classifier(feature_dic).view(1, MAX_COL_COUNT, -1)
    mask = mask.view(1, MAX_COL_COUNT)
    pred = model.decode(emissions, mask)[0]

    return label_enc.inverse_transform(pred)

In [16]:
predicted_labels = [] #pd.DataFrame(columns=['type'])
write_out_limit = 2000

In [17]:
write_out_idx = 0
for idx, col in enumerate(validation_values['values']):
    col = ast.literal_eval(col)
    df = pd.DataFrame(columns=['col'], data=col)
    prediction = evaluate(df)
    predicted_labels.extend(prediction)
    df = None
#     true_types.extend(df_header)
    if((idx+1) % 100 == 0):
        print(idx)
    if ((idx+1) % write_out_limit == 0):
        # write out predictions to parquet file
        print("writing out predictions...")
        df_pred = pd.DataFrame(data=predicted_labels, columns=['predicted_type'])
        # write out predictions, format: model_dataset_tableCount_writeOutCount.parquet
        df_pred.to_parquet(join(path_out_temp, 'sato_sherlock_'+ str(idx) + '_' +str(write_out_idx)+ '.parquet'))
        write_out_idx += 1
        predicted_labels = []
        df_pred = None
            
df_pred = pd.DataFrame(predicted_labels, columns=['predicted_type'])
df_pred.to_parquet(join(path_out_temp, 'sato_sherlock_'+ str(idx) + '_' +str(write_out_idx)+ '.parquet'))


/home/senn/virtualenvs/col2type/lib/python3.7/site-packages/pandas/core/strings/object_array.py:90: FutureWarning: Possible nested set at position 1
  regex = re.compile(pat, flags=flags)
/home/senn/virtualenvs/col2type/lib/python3.7/site-packages/pandas/core/strings/object_array.py:90: FutureWarning: Possible nested set at position 1
  regex = re.compile(pat, flags=flags)


122099
122199
122299
122399
122499
122599
122699
122799
122899
122999
123099
123199
123299
123399
123499
123599
123699
123799
123899
123999
writing out predictions...
124099
124199
124299
124399
124499
124599
124699
124799
124899
124999
125099
125199
125299
125399
125499
125599
125699
125799
125899
125999
writing out predictions...
126099
126199
126299
126399
126499
126599
126699
126799
126899
126999
127099
127199
127299
127399
127499
127599
127699
127799
127899
127999
writing out predictions...
128099
128199
128299
128399
128499
128599
128699
128799
128899
128999
129099
129199
129299
129399
129499
129599
129699
129799
129899
129999
writing out predictions...
130099
130199
130299
130399
130499
130599
130699
130799
130899
130999
131099
131199
131299
131399
131499
131599
131699
131799
131899
131999
writing out predictions...
132099
132199
132299
132399
132499
132599
132699
132799
132899
132999
133099
133199
133299
133399
133499
133599
133699
133799
133899
133999
writing out predictions..

In [4]:
files = listdir(path_out_temp)
files.sort(key=lambda x: int(x.split('_')[2]))
# print(files)
df_combined = pd.DataFrame()
for file in files:
    filepath = os.path.join(path_out_temp, file)
    df_partial = pd.read_parquet(filepath)
    df_combined = pd.concat([df_combined, df_partial], ignore_index=True, sort=False)

df_combined.columns = ['type']
print(df_combined.tail())

            type
137348      name
137349   address
137350  duration
137351     class
137352      name


In [19]:
# print(predicted_labels)

In [20]:
# df = pd.DataFrame(columns=['type'], data=predicted_labels)

In [5]:
df_combined.to_parquet(join(path_out_predictions, "sato_sherlock_validation_137353.parquet"))
# df_combined.to_parquet(join(path_out_predictions, "sato_sherlock_validation.parquet"))